In [1]:
import struct

def float_to_bits(f):
    """32비트 float → 이진 문자열"""
    [d] = struct.unpack(">I", struct.pack(">f", f))
    return f"{d:032b}"

def bits_to_float(bstr):
    """이진 문자열 → 32비트 float"""
    i = int(bstr, 2)
    [f] = struct.unpack(">f", struct.pack(">I", i))
    return f

def decode_float(bstr):
    """부동소수점 구성요소 추출"""
    sign = int(bstr[0], 2)
    exponent = int(bstr[1:9], 2)
    mantissa = bstr[9:]
    bias = 127
    actual_exp = exponent - bias
    mantissa_val = 1 + sum([int(bit)*2**(-i-1) for i, bit in enumerate(mantissa)])
    return sign, exponent, mantissa_val, actual_exp

# 1️⃣ 두 수의 32비트 표현
a, b = 0.75, 0.25
bits_a = float_to_bits(a)
bits_b = float_to_bits(b)

print("A = 0.75")
print("비트 표현:", bits_a)
print("B = 0.25")
print("비트 표현:", bits_b)

# 2️⃣ 각 요소 분리
sign_a, exp_a, mant_a, act_exp_a = decode_float(bits_a)
sign_b, exp_b, mant_b, act_exp_b = decode_float(bits_b)

print("\n[A 해석]")
print(f"부호: {sign_a}, 지수(실제): {act_exp_a}, 가수: {mant_a}")
print("[B 해석]")
print(f"부호: {sign_b}, 지수(실제): {act_exp_b}, 가수: {mant_b}")

# 3️⃣ 지수 맞추기 (exponent alignment)
if exp_a > exp_b:
    shift = exp_a - exp_b
    mant_b /= 2**shift
    exp_res = exp_a
elif exp_b > exp_a:
    shift = exp_b - exp_a
    mant_a /= 2**shift
    exp_res = exp_b
else:
    exp_res = exp_a

# 4️⃣ 가수 덧셈
mant_res = mant_a + mant_b

# 5️⃣ 정규화
if mant_res >= 2:
    mant_res /= 2
    exp_res += 1

# 6️⃣ 결과 계산
bias = 127
exp_bits = f"{exp_res:08b}"
mant_bits = ""
m_temp = mant_res - 1
for i in range(23):
    m_temp *= 2
    bit = int(m_temp)
    mant_bits += str(bit)
    m_temp -= bit

result_bits = f"0{exp_bits}{mant_bits}"
result_value = bits_to_float(result_bits)

print("\n[연산 과정 요약]")
print(f"정렬 후 가수 합: {mant_a:.6f} + {mant_b:.6f} = {mant_res:.6f}")
print(f"최종 지수: {exp_res}  (bias={bias})")
print("\n결과 비트:", result_bits)
print("IEEE754 해석 값 =", result_value)


A = 0.75
비트 표현: 00111111010000000000000000000000
B = 0.25
비트 표현: 00111110100000000000000000000000

[A 해석]
부호: 0, 지수(실제): -1, 가수: 1.5
[B 해석]
부호: 0, 지수(실제): -2, 가수: 1.0

[연산 과정 요약]
정렬 후 가수 합: 1.500000 + 0.500000 = 1.000000
최종 지수: 127  (bias=127)

결과 비트: 00111111100000000000000000000000
IEEE754 해석 값 = 1.0


In [2]:
import struct

def float_to_bits(f):
    """32비트 float → 이진 문자열"""
    [d] = struct.unpack(">I", struct.pack(">f", f))
    return f"{d:032b}"

def bits_to_float(bstr):
    """이진 문자열 → 32비트 float"""
    i = int(bstr, 2)
    [f] = struct.unpack(">f", struct.pack(">I", i))
    return f

def decode_float(bstr):
    """부동소수점 구성요소 추출"""
    sign = int(bstr[0], 2)
    exponent = int(bstr[1:9], 2)
    mantissa_bits = bstr[9:]
    bias = 127
    actual_exp = exponent - bias
    mantissa_val = 1 + sum(int(bit)*2**(-i-1) for i, bit in enumerate(mantissa_bits))
    return sign, exponent, mantissa_val, actual_exp

def encode_float(sign, exponent, mantissa):
    """sign, exponent, mantissa → IEEE754 32비트 문자열"""
    bias = 127
    e_bits = f"{exponent:08b}"
    m_val = mantissa - 1
    m_bits = ""
    for i in range(23):
        m_val *= 2
        bit = int(m_val)
        m_bits += str(bit)
        m_val -= bit
    return f"{sign}{e_bits}{m_bits}"

def normalize(mant, exp):
    """가수를 1.xxx 형태로 정규화"""
    while mant >= 2:
        mant /= 2
        exp += 1
    while mant < 1 and exp > 0:
        mant *= 2
        exp -= 1
    return mant, exp

def simulate_add(a, b):
    """덧셈"""
    sa, ea, ma, exa = decode_float(float_to_bits(a))
    sb, eb, mb, exb = decode_float(float_to_bits(b))

    # 지수 정렬
    if ea > eb:
        mb /= 2**(ea-eb)
        e_res = ea
    else:
        ma /= 2**(eb-ea)
        e_res = eb

    # 부호 같다고 가정 (+,+)
    m_res = ma + mb
    m_res, e_res = normalize(m_res, e_res)

    result_bits = encode_float(0, e_res, m_res)
    return bits_to_float(result_bits), result_bits, m_res, e_res

def simulate_mul(a, b):
    """곱셈"""
    sa, ea, ma, exa = decode_float(float_to_bits(a))
    sb, eb, mb, exb = decode_float(float_to_bits(b))
    s_res = sa ^ sb
    e_res = ea + eb - 127
    m_res = ma * mb
    m_res, e_res = normalize(m_res, e_res)
    result_bits = encode_float(s_res, e_res, m_res)
    return bits_to_float(result_bits), result_bits, m_res, e_res

def simulate_div(a, b):
    """나눗셈"""
    sa, ea, ma, exa = decode_float(float_to_bits(a))
    sb, eb, mb, exb = decode_float(float_to_bits(b))
    s_res = sa ^ sb
    e_res = ea - eb + 127
    m_res = ma / mb
    m_res, e_res = normalize(m_res, e_res)
    result_bits = encode_float(s_res, e_res, m_res)
    return bits_to_float(result_bits), result_bits, m_res, e_res


# 실험값
a, b = 0.75, 0.25
print("===== IEEE 754 부동소수점 연산 시뮬레이터 =====")
print(f"A = {a}, B = {b}")
print("-----------------------------------------------")

# 덧셈
r_add, bits_add, m_add, e_add = simulate_add(a, b)
print("[+] 덧셈 (A + B)")
print(f"가수합 = {m_add:.6f}, 지수 = {e_add}")
print("결과 비트:", bits_add)
print("결과 값 =", r_add)
print("-----------------------------------------------")

# 곱셈
r_mul, bits_mul, m_mul, e_mul = simulate_mul(a, b)
print("[×] 곱셈 (A × B)")
print(f"가수곱 = {m_mul:.6f}, 지수 = {e_mul}")
print("결과 비트:", bits_mul)
print("결과 값 =", r_mul)
print("-----------------------------------------------")

# 나눗셈
r_div, bits_div, m_div, e_div = simulate_div(a, b)
print("[÷] 나눗셈 (A ÷ B)")
print(f"가수나눗셈 = {m_div:.6f}, 지수 = {e_div}")
print("결과 비트:", bits_div)
print("결과 값 =", r_div)


===== IEEE 754 부동소수점 연산 시뮬레이터 =====
A = 0.75, B = 0.25
-----------------------------------------------
[+] 덧셈 (A + B)
가수합 = 1.000000, 지수 = 127
결과 비트: 00111111100000000000000000000000
결과 값 = 1.0
-----------------------------------------------
[×] 곱셈 (A × B)
가수곱 = 1.500000, 지수 = 124
결과 비트: 00111110010000000000000000000000
결과 값 = 0.1875
-----------------------------------------------
[÷] 나눗셈 (A ÷ B)
가수나눗셈 = 1.500000, 지수 = 128
결과 비트: 01000000010000000000000000000000
결과 값 = 3.0


In [3]:
struct.pack(">f", 0.75)

b'?@\x00\x00'

In [4]:
[d] = struct.unpack(">I", b'?@\x00\x00')
d

1061158912

In [5]:
f"{d:032b}"

'00111111010000000000000000000000'